In [1]:
import pandas as pd
from sdv.single_table import TVAESynthesizer

from domias.evaluator import evaluate_performance
from domias.models.generator import GeneratorInterface
from domias.models.ctgan import CTGAN



from scipy import stats
from scipy.stats import multivariate_normal


import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing, fetch_covtype, load_digits

import warnings
warnings.filterwarnings("ignore")

[KeOps] Warning : 
    The default C++ compiler could not be found on your system.
    You need to either define the CXX environment variable or a symlink to the g++ command.
    For example if g++-8 is the command you can do
      import os
      os.environ['CXX'] = 'g++-8'
    
[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [3]:
#Read in UCI dataset
df_uci = pd.read_csv(r'C:\Users\jordy\OneDrive\MSc_Python\Individual_Project\Data\loan.csv', index_col=0)
df_uci


Loan Amount  Funded Amount  Funded Amount Investor  Term  \
0            10000          32236             12329.36286    59   
1             3609          11940             12191.99692    59   
2            28276           9311             21603.22455    59   
3            11170           6954             17877.15585    59   
4            16890          13226             13539.92667    59   
...            ...            ...                     ...   ...   
67458        13601           6848             13175.28583    59   
67459         8323          11046             15637.46301    59   
67460        15897          32921             12329.45775    59   
67461        16567           4975             21353.68465    59   
67462        15353          29875             14207.44860    59   

       Batch Enrolled  Interest Rate  Grade  Sub Grade  Employment Duration  \
0                  16      11.135007      1         13                    0   
1                   4      12.237563      2         17                    2   
2                  11      12.545884      5         18                    0   
3                  15      16.731201      2         12                    0   
4                  32      15.008300      2         18                    0   
...               ...            ...    ...        ...                  ...   
67458              21       9.408858      2          3                    0   
67459               7       9.972104      2          7                    2   
67460               5      19.650943      0         27                    0   
67461              14      13.169095      3         22                    1   
67462               8      16.034631      1         15                    0   

       Home Ownership  ...  Total Received Late Fee   Recoveries  \
0        176346.62670  ...                 0.102055     2.498291   
1         39833.92100  ...                 0.036181     2.377215   
2         91506.69105  ...                18.778660     4.316277   
3        108286.57590  ...                 0.044131     0.107020   
4         44234.82545  ...                19.306646  1294.818751   
...               ...  ...                      ...          ...   
67458     83961.15003  ...                 0.023478   564.614852   
67459     65491.12817  ...                 0.027095     2.015494   
67460     34813.96985  ...                 0.028212     5.673092   
67461     96938.83564  ...                 0.074508     1.157454   
67462    105123.15580  ...                 0.000671     1.856480   

       Collection Recovery Fee  Collection 12 months Medical  \
0                     0.793724                             0   
1                     0.974821                             0   
2                     1.020075                             0   
3                     0.749971                             0   
4                     0.368953                             0   
...                        ...                           ...   
67458                 0.865230                             0   
67459                 1.403368                             0   
67460                 1.607093                             0   
67461                 0.207608                             0   
67462                 0.366386                             0   

       Application Type  Last week Pay  Total Collection Amount  \
0                     0             49                       31   
1                     0            109                       53   
2                     0             66                       34   
3                     0             39                       40   
4                     0             18                      430   
...                 ...            ...                      ...   
67458                 0             69                       48   
67459                 0             14                       37   
67460                 0            137                       17 

In [3]:
#Define data loader
def get_dataset() -> np.ndarray:
    def data_loader() -> np.ndarray:
        scaler = StandardScaler()
        X =arr_uci
        np.random.shuffle(X)
        return scaler.fit_transform(X)

    return data_loader()

In [4]:
def get_generator(
    gan_method: str = "CTGAN",
    epochs: int = 1000,
    seed: int = 0,
) -> GeneratorInterface:
    class LocalGenerator(GeneratorInterface):
        def __init__(self) -> None:
            if gan_method == "TVAE":
                syn_model = TVAESynthesizer(metadata, epochs=epochs)
            elif gan_method == "CTGAN":
                syn_model = CTGAN(epochs=epochs)
            elif gan_method == "KDE":
                syn_model = None
            else:
                raise RuntimeError()
            self.method = gan_method
            self.model = syn_model
                

        def fit(self, data: pd.DataFrame) -> "LocalGenerator":
            if self.method == "KDE":
                self.model = stats.gaussian_kde(np.transpose(data))
            else:
                self.model.fit(data)
            return self

        def generate(self, count: int) -> pd.DataFrame:
            
            if gan_method == "KDE":
                samples = pd.DataFrame(self.model.resample(count).transpose(1, 0))
            elif gan_method == "TVAE":
                samples = self.model.sample(count)
            elif gan_method == "CTGAN":
                samples = self.model.generate(count)
            else:
                raise RuntimeError()

            return samples
            
            #return self.model.sample(count)

    return LocalGenerator()


#Loading metadata from dataset for use in TVAESynthesizer
dataset = get_dataset()
# df_dataset = pd.DataFrame(dataset)
# df_dataset.rename(columns={0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7'}, inplace = True)

# from sdv.metadata import SingleTableMetadata

# metadata = SingleTableMetadata()
# metadata.detect_from_dataframe(data=df_dataset)






"""
Args:
    generator: GeneratorInterface
        Generator with the `fit` and `generate` methods. The generator MUST not be fitted.
    dataset: int
        The evaluation dataset, used to derive the training and test datasets.
    training_size: int
        The split for the training (member) dataset out of `dataset`
    reference_size: int
        The split for the reference dataset out of `dataset`.
    training_epochs: int
        Training epochs
    synthetic_sizes: List[int]
        For how many synthetic samples to test the attacks.

"""

"""
CTGAN Args:

embedding_dim: int = 128,
generator_dim: Tuple = (256, 256),
discriminator_dim: Tuple = (256, 256),
generator_lr: float = 2e-4,
generator_decay: float = 1e-6,
discriminator_lr: float = 2e-4,
discriminator_decay: float = 1e-6,
batch_size: int = 500,
discriminator_steps: int = 1,
log_frequency: bool = True,
verbose: bool = False,
epochs: int = 300,
pac: int = 1,
cuda: bool = True,



"""


# mem_set_size = 1000 -> originally what training size was
reference_set_size = 10000 #held out set
training_epochs = [100, 500, 1000, 2000, 3000, 5000]
training_sizes = [10000]
#synthetic_sizes = [200]
density_estimator = "kde"  # prior, kde, bnaf
gen_size = 500 #same as synthetic_sizes

method = "CTGAN"

# Create a dictionary to store the results
results = {}

# Set the number of iterations
num_iterations = 5

for iteration in range(1, num_iterations+1):
    # Initialize the result dictionary for the current iteration
    iteration_results = {}
    
    for training_size in training_sizes:
        # Initialize the result dictionary for the current training size
        size_results = {}
        
        for training_epoch in training_epochs:
            generator = get_generator(
                gan_method=method,
                epochs=training_epoch,
            )
    
            perf = evaluate_performance(
                generator,
                dataset,
                training_size,
                reference_set_size,
                training_epochs=training_epoch,
                synthetic_sizes=[gen_size],
                density_estimator=density_estimator,
            )
            
            # Store the MIA performance for the current training size and epoch
            size_results[training_epoch] = perf[gen_size]["MIA_performance"]
        
        # Store the results for the current training size
        iteration_results[training_size] = size_results
    
    # Store the results for the current iteration
    results[iteration] = iteration_results

# Print the results
for iteration, iteration_results in results.items():
    print(f"Iteration {iteration}:")
    for training_size, size_results in iteration_results.items():
        print(f"Training Size {training_size}:")
        for training_epoch, mia_performance in size_results.items():
            print(f"Training Epoch {training_epoch}: MIA Performance = {mia_performance}")
        print()


p_G_evaluated
[1.06373935e-06 1.44551472e-10 1.73357588e-15 ... 2.61694379e-07
 7.62717311e-09 9.31029338e-09]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0    0.375623  0.812997  0.179815 -1.052557 -1.281391 -1.738438 -1.563210   
1    1.782134 -1.247751  0.177818  0.878134 -0.906486  1.819009  2.729716   
2    1.300818  0.805588  0.177267 -1.063757  0.383946 -1.739479 -1.553795   
3    1.365974  0.813515 -1.096226 -1.053320  0.742203 -1.703435 -1.574918   
4   -0.963554  0.816861  1.460343  0.863230 -1.193271  0.026352  0.107885   
..        ...       ...       ...       ...       ...       ...       ...   
195 -0.905914  0.821962 -1.083415 -1.045947  0.296575 -0.862464 -0.746903   
196  2.305389  0.797893  0.182960  0.868533 -1.408484  1.784328  0.109718   
197 -0.720063  0.794958  0.177632 -1.051215 -0.590149  0.032639  0.118727  

p_G_evaluated
[1.46846970e-06 8.23467299e-09 1.87362697e-08 ... 2.21479981e-06
 1.22164145e-07 1.00708423e-08]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0    0.076526  0.812890  0.184767 -1.063315  2.564438 -0.874412 -0.720784   
1   -0.721717  0.809450  0.185977  0.855209 -0.800360  0.905034 -0.721140   
2   -0.873789 -1.235741 -1.079128  0.855212 -1.092255  0.014740  0.111794   
3   -0.411500  0.811102  0.184378  0.853749 -0.615111  0.900069  1.777492   
4   -0.962903  0.811864  0.184929  0.855056 -0.458937  0.899774 -1.555933   
..        ...       ...       ...       ...       ...       ...       ...   
195 -0.789469  0.811039  1.449538 -1.060360  1.408060 -0.870734 -0.724377   
196 -0.378518  0.811289 -1.077189  0.855255 -0.256928  0.016236  0.114665   
197  0.094480  0.811332  0.185979  0.857444 -0.251348 -1.776359 -1.565284  

p_G_evaluated
[1.70857485e-07 7.50207237e-08 1.31807019e-15 ... 1.13040005e-06
 2.47027808e-08 2.65909478e-07]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0   -1.179518  0.801436  1.450733  0.840019 -0.153500 -0.880772 -0.723286   
1   -0.830790 -1.240145  0.174821 -1.070152 -0.149459  0.008518  0.115068   
2   -0.153070  0.820382  0.195066  0.855567 -1.244885  2.778547  1.765061   
3    0.361401  0.819519  1.435936 -1.058839 -0.865605  0.005844  0.122992   
4   -0.234655 -1.233457  0.174019  0.848891 -0.543103  0.018497  0.128093   
..        ...       ...       ...       ...       ...       ...       ...   
195 -0.886890  0.817947  1.423266  0.845371  0.661234  1.725797  1.801786   
196  0.425193 -1.233510 -1.087137  0.861543 -1.172671  0.019783  0.108399   
197  1.100619 -1.224160  1.394327  0.846913 -0.370921  0.017018  0.104032  

p_G_evaluated
[8.19671331e-07 1.65508555e-08 5.63613461e-12 ... 3.28441056e-09
 8.47987207e-09 6.49551863e-13]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0    0.087336 -1.231806  0.184487 -1.055574  1.027628  0.015489  0.113336   
1   -0.344764  0.810746 -1.082315  0.863156 -0.740881 -1.769205 -1.563306   
2   -0.926577  0.811747  1.440401 -1.057300  0.403630  0.014710  0.110176   
3    0.141794  0.810037 -1.080451 -1.057986  1.540482  0.015346  0.112754   
4   -0.751984  0.812124  0.183764  0.860903 -0.901930  0.016909  0.113762   
..        ...       ...       ...       ...       ...       ...       ...   
195  0.214741  0.811433  0.184332  0.860195 -0.666388  0.902375 -0.724387   
196 -0.928499  0.809852  0.184469  0.862322 -1.249323  0.017997  0.112899   
197  0.957636  0.812225  0.187255  0.858127  0.182671 -1.757890 -0.719628  

p_G_evaluated
[1.22419864e-07 1.33663109e-07 6.01110967e-11 ... 3.06468120e-07
 1.64495184e-08 1.02509462e-10]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0   -0.875860  0.802950  0.194223  0.852665  1.713102  0.029963  0.104546   
1    1.335298 -1.225685 -1.050315 -1.053714  2.009316 -1.724128 -1.587408   
2    0.274631 -1.226416  0.183766 -1.053339 -0.333944 -1.761055 -1.571277   
3    1.055773 -1.238960  0.190988  0.854520  0.355828  0.013161  0.109238   
4    0.517126  0.814206  3.926262 -1.059065  0.788187  0.016622  0.104857   
..        ...       ...       ...       ...       ...       ...       ...   
195 -0.895272  0.802489  0.190559  0.850796 -0.461029  0.023245  0.108007   
196 -0.537769  0.804087  1.479132 -1.065355  1.652080 -0.867841 -0.726330   
197 -1.114530 -1.243090  0.191908  0.859292 -0.062049  0.023618  1.779507  

p_G_evaluated
[2.97645126e-07 2.66555186e-09 4.22599514e-10 ... 4.34741198e-07
 2.32679162e-08 1.35702422e-10]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0   -0.971392  0.811190  1.460439 -1.061937  2.776388  0.016982  0.110869   
1   -0.720005  0.810881  0.189385 -1.060701  0.318790  0.015449  0.108551   
2   -0.009221 -1.232364  0.188516  0.859390 -0.542134  0.906181 -0.734700   
3    0.188043 -1.233143 -1.074094 -1.056890  0.017679 -1.752259 -1.564092   
4   -0.419175  0.811573  0.189739 -1.063542 -0.681448  0.906237  1.780792   
..        ...       ...       ...       ...       ...       ...       ...   
195  0.928581  0.810262  0.186674 -1.059626  0.253573 -0.873978 -0.728080   
196  0.023252  0.812793  0.185978 -1.060550  1.193317 -0.871530 -0.727481   
197 -1.199260  0.810883  0.187497 -1.061003  1.038430  0.915240  2.965286  

p_G_evaluated
[4.06723789e-09 1.56579312e-09 2.50013941e-10 ... 2.00755792e-08
 6.72830506e-09 1.52992533e-10]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0   -0.861612  0.820622 -1.073103  0.870886 -0.493574  1.857067  1.759758   
1   -0.740869  0.818469  0.194095  0.858534 -0.535960  0.016490  0.109249   
2   -1.233616  0.813392  0.196445  0.875610 -0.994250 -0.862932 -0.718242   
3   -0.928470 -1.237328  1.480618  0.871260 -1.016240  0.019236  0.113280   
4   -1.168598  0.806018  0.191351  0.865825 -0.326422  1.766111  0.118406   
..        ...       ...       ...       ...       ...       ...       ...   
195 -0.635312  0.810028  0.193023  0.866646 -0.477990  0.024797 -0.726100   
196 -0.754414 -1.258355  0.192802 -1.035801  0.335786  0.019470  0.114549   
197 -0.444859 -1.224732  1.450313  0.860519  2.731422  0.024523  0.116218  

p_G_evaluated
[2.08460401e-07 2.77556158e-07 3.70270733e-10 ... 2.30858848e-07
 1.21568964e-06 3.92598748e-08]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0    2.451565 -1.232197 -1.077930 -1.056559 -0.618798 -0.873946 -0.723691   
1   -0.839648 -1.235408  0.185394  0.861297 -0.577527  0.012945  0.110485   
2    1.845502  0.810138 -1.076690 -1.059841  2.023932  1.784500  1.786099   
3   -0.013805  0.809736  0.185835  2.752034  2.710497  0.011832  0.110551   
4    1.762429 -1.231379  0.187388 -1.056080  1.542311 -0.878386 -1.563550   
..        ...       ...       ...       ...       ...       ...       ...   
195  0.361731 -1.233325  0.186825  0.860906  1.537215  0.015029  0.111806   
196  1.660857  0.810953  1.447929 -1.056753 -0.358029  0.016515  0.111204   
197  2.617276 -1.231974 -1.079935 -1.056191  0.493347 -0.877557 -0.726248  

p_G_evaluated
[7.92502233e-08 5.13809631e-10 2.07935971e-14 ... 1.66945095e-08
 1.60194968e-09 4.75062251e-13]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0   -0.990302  0.795517  0.185812 -1.042854 -0.176799  3.010166  1.748643   
1   -0.566104  0.808953  0.199427  0.876616 -1.197108 -0.870874 -0.728076   
2   -1.209657  0.800676  0.192951  0.873494 -1.543592  0.915464  1.759180   
3   -1.002832 -1.233503 -1.048373  0.877366 -0.629262  0.899325  0.092915   
4    0.331246  0.808268  0.197621  0.878289  0.886874 -0.862760 -0.721370   
..        ...       ...       ...       ...       ...       ...       ...   
195 -0.524512  0.795979  0.202586  0.873930 -0.273449 -0.865719 -0.739619   
196 -0.005449 -1.252286  0.199462 -1.023212 -1.347725 -1.750090 -1.621251   
197  0.278201  0.796132  0.198320  0.875032  0.725884 -1.778645 -0.738691  

p_G_evaluated
[3.51561289e-06 1.54486708e-06 4.35871537e-11 ... 1.28492349e-07
 3.31027781e-08 3.42819755e-11]
p_R_evaluated
[6.89169339e-05 5.76685916e-06 2.01421229e-11 ... 7.91361405e-06
 2.63904439e-06 1.08091335e-07]
Synth_set
            0         1         2         3         4         5         6  \
0   -0.868113 -1.234653 -1.072869  0.858017 -0.849420  1.811471  1.785392   
1    0.164124  0.811901  1.460589 -1.053915  1.567924  0.019731  0.113529   
2   -1.265997  0.810216  1.465666  0.857569 -0.993815  0.907473  1.775509   
3   -1.218716 -1.235567  0.187434 -1.053799 -0.876483  0.015551  0.109157   
4   -0.497637  0.810217  1.453821  0.860375 -0.734054  0.016905  0.111210   
..        ...       ...       ...       ...       ...       ...       ...   
195 -0.613924  0.809567  1.457415 -1.052614  2.359278  0.018632  0.112699   
196 -0.167759 -1.234195 -1.071923  0.863012 -0.448182  0.018696  0.112678   
197 -0.349567  0.809723  0.189303 -1.053478  1.240540 -1.752973  1.778655  

In [5]:
reference_set_size = reference_set_size
training_size = training_sizes[0]
gen_size = gen_size

epoch_output = pd.DataFrame([], columns=["iteration", "epoch", "training_size", "reference_size", "gen_size", "src", "aucroc"])

for iteration in range(1, num_iterations + 1):
    for training_epoch in training_epochs:
        epoch_res = results[iteration][training_size][training_epoch]
        perf = epoch_res  # Assuming perf is the correct value for accuracy

        for key in perf:
            epoch_output = pd.concat(
                [
                    epoch_output,
                    pd.DataFrame(
                        [
                            [iteration, training_epoch, training_size, reference_set_size, gen_size, key, perf[key]["aucroc"]]
                        ],
                        columns=["iteration", "epoch", "training_size", "reference_size", "gen_size", "src", "aucroc"],
                    ),
                ]
            )
epoch_output

iteration epoch training_size reference_size gen_size,            src  \
0          1   100         10000          10000       200    ablated_eq1   
0          1   100         10000          10000       200    ablated_eq2   
0          1   100         10000          10000       200       LOGAN_D1   
0          1   100         10000          10000       200             MC   
0          1   100         10000          10000       200      gan_leaks   
..       ...   ...           ...            ...       ...            ...   
0          5  5000         10000          10000       200      gan_leaks   
0          5  5000         10000          10000       200  gan_leaks_cal   
0          5  5000         10000          10000       200        LOGAN_0   
0          5  5000         10000          10000       200            eq1   
0          5  5000         10000          10000       200         domias   

      aucroc  
0   0.503995  
0   0.502323  
0   0.497447  
0   0.501412  
0   0.503534  
..       ...  
0   0.505545  
0   0.506210  
0   0.506346  
0   0.504719  
0   0.503982  

[270 rows x 7 columns]

In [6]:
#results df to csv
epoch_output.to_csv('kde_epoch_output_loan.csv')